In [1]:
#import dependecies
import time
from IPython.display import clear_output
import pandas as pd
from datetime import datetime
from config import weather_api_key
from config import g_key
from citipy import citipy
import numpy as np
import requests


In [2]:
#create random latitudes and longitudes
lats = np.random.uniform(low=-90.000,high= 90.000, size=2000)
lngs = np.random.uniform(low=-180.000,high=180.000, size=2000)

In [3]:
#list comprehension used with citipy module to iterate through random lats & lngs to create a list of tuples containing the city name and the country name
city_names_country_code=[(citipy.nearest_city(i,v).city_name,citipy.nearest_city(i,v).country_code) for i,v in zip(lats,lngs)]

In [4]:
 #for i,v in city_names_country_code:print(i,v)

In [5]:
#query API (start query)
print("Beginning Data Retrieval")
print("-"*30)

#accumulator to build Pandas DataFrame
world_data=[]

#index for record count / percentage
index=0
counter=0
#iterate through list of tuples containing city_names (i) and country_code (v) 
for i,v in city_names_country_code:
    
    #keep track of number of loops with index
    index+=1
    
    #create a dynamic url that queries based on city_name (i)
    url="https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+weather_api_key+"&q="+i
    
    #use the request library to request the data for the current url
    response=requests.get(url)
    
    #convert the reponse into a json object/dictionary
    json_dict=response.json()
    
    #utilize try/except to check the currect json object for our wanted keys and values, saved to a variable without throwing an error if key does not exist in object
    try:
        city_lat = json_dict["coord"]["lat"]
        city_lng = json_dict["coord"]["lon"]
        city_max_temp = json_dict["main"]["temp_max"]
        city_humidity = json_dict["main"]["humidity"]
        city_clouds = json_dict["clouds"]["all"]
        city_wind = json_dict["wind"]["speed"]
        weather_description = json_dict["weather"][0]["description"]
        city_country = json_dict["sys"]["country"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(json_dict["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into world_data list.
        world_data.append({"City": i,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max_Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind_Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Weather_Description":weather_description})
        
        #After the data is succesfully collected print the following message: city, country, iteration (index), and percentage (index/len(city_names_country_code))
        print(f'city:    {i.title()}\ncountry: {v.upper()}\n\t{response} {index} of {len(city_names_country_code)} | {"{:.2f}".format((index/len(city_names_country_code)*100))}% completed\n')
        counter+=1
        if counter == 10:
            clear_output()
            counter=0
    #KeyError would reflect that the json object does not contain the information we are looking for. we catch and skip to the next object
    except KeyError:
        print(f"\n\nWeather data for City {i.title()} in Country {v.upper()} not found. Skipping...\n\n")
        counter+=1
        if counter == 10:
            clear_output()
            counter=0
        pass
        
print("-"*30)
print("Data Retrieval Complete      ")
print("-"*30)


------------------------------
Data Retrieval Complete      
------------------------------


In [6]:
#create a pandas DataFrame from the list of dictionarys/ array of json objects
world_temp_data_by_city = pd.DataFrame(world_data)
#display part of collected data in the DataFrame
world_temp_data_by_city.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Weather_Description
0,upernavik,72.79,-56.15,20.95,92,100,7.56,GL,2020-10-24 08:17:49,overcast clouds
1,bambous virieux,-20.34,57.76,80.60,54,75,13.87,MU,2020-10-24 08:17:49,broken clouds
2,state college,40.79,-77.86,61.00,87,75,5.82,US,2020-10-24 08:14:25,moderate rain
3,castro,-24.79,-50.01,57.76,95,100,5.14,BR,2020-10-24 08:16:04,overcast clouds
4,rikitea,-23.12,-134.97,74.07,83,100,14.29,PF,2020-10-24 08:17:50,light rain


In [7]:
#export Data to a csv
world_temp_data_by_city.to_csv("WeatherPy_Database.csv")
